In [ ]:
import os
import statistics
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from itertools import product
from skimage import measure
from math import sqrt
from sklearn.cluster import KMeans
from scipy import ndimage
from skimage import io
from scipy.stats import pearsonr

In [ ]:
def dpm_3d(
    ground_truth: int, prediction: int, 
    size_threshold_low: int, size_threshold_up: int, scale: float,
    weight_d, weight_s, weight_a):
    '''
    Returns the discrete protein metric (DPM) similarity score of two focal adhesion (FA) images.
    
            Parameters:
                    ground_truth: A numpy array of stack containing membrane and ground truth FA
                    prediction: A numpy array of stack containing membrane and predicted FA to be compared to ground truth
                    size_threshold_low: An integer value for lower bound size threshold; any FAs with an area less than this value are dropped
                    size_threshold_up: An integer value for upper bound size threshold; any FAs with an area higher than this value are dropped
                    scale: A float value for image scale (microns/pixel)
                    weight_d: A float value for importance weight to be applied for distribution measurement
                    weight_s: A float value for importance weight to be applied for shape/size measurement
                    weight_a: A float value for importance weight to be applied for angle measurement
            
            Returns:
                    dpm: A numpy float value for the discrete protein metric score
                    d: A numpy float value for the distribution measurement
                    s: A numpy float value for the shape/size measurement
                    a: A numpy float value for the angle measurement
                    fa_binary_truth: A numpy array of the 2D FA projection ground truth image
                    fa_binary_pred: A numpy array of the 2D FA projection predicted image
    '''
    
    def euclidean_distance(row1: slice, row2: slice):
        '''
        Returns the euclidean distance between two coordinate points.
        
            Parameters:
                    row1: A slice of a list containing a single coordinate point
                    row2: A slice of a list containing another coordinate point
            
            Returns:
                    sqrt(distance): A float value for the euclidean distance
        '''
        distance = 0.0
        for i in range(len(row1)-1):
            distance += (row1[i] - row2[i])**2
        return sqrt(distance)

    
    def get_neighbors(train: list, test_row: slice, num_neighbors: int):
        '''
        Returns the n nearest neighbors for one coordinate point.
        
            Parameters:
                    train: A list containing a set of coordinate points
                    test_row: A slice of a list containing a single coordinate point
                    num_neighbors: An integer value for the n nearest neighbors to be retreived
            
            Returns:
                    neighbors: A list of coordinate points for the n nearest neighbors
        '''
        distances = list()
        for train_row in train:
            dist = euclidean_distance(test_row, train_row)
            distances.append((train_row, dist))
        distances.sort(key=lambda tup: tup[1])
        neighbors = list()
        for i in range(num_neighbors):
            neighbors.append(distances[i][0])
        return neighbors

    
    def fa_processing_stack(img: int, size_threshold_low: int, size_threshold_up: int, scale: float):
        '''
        Returns numpy arrays for the binary and segmented FA images from the raw FA images.

            Parameters:
                    img: A numpy array of stack containing raw FA
                    size_threshold_low: An integer value for lower bound size threshold; any FAs with an area less than this value are dropped
                    size_threshold_up: An integer value for upper bound size threshold; any FAs with an area higher than this value are dropped
                    scale: A float value for image scale (microns/pixel)

            Returns:
                    new_stack: A numpy array of the 2D projection of the processed FA stack
        '''
        # Read image and extract FA channel into a new empty image
        stack = img
        new_stack = np.zeros((256, 256), dtype=np.uint8)
        for i in range(len(stack)):
            # Read single slice and extract to empty image
            img = stack[i]
            res = np.zeros((256, 256, 3))
            res[:,:,1] = img[:,:,1]
            # Apply a top hat filter with 3x3 rectangular kernel and place filtered image into a new empty image
            filterSize =(3, 3)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize)
            tophat_img = cv2.morphologyEx(res[:,:,1], cv2.MORPH_TOPHAT, kernel)        
            processed_img = res
            processed_img[:,:,1] = tophat_img
            # Binarize filtered FA image
            ret, bw_img = cv2.threshold(processed_img,10,255,cv2.THRESH_BINARY)
            gray = rgb2gray(bw_img)
            gray = gray * 255
            gray = gray.astype(np.uint8)
            ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
            # Find the contours of each FA
            contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            binary_img = np.zeros((256,256), dtype=np.uint8)
            # Iterate through each contour
            for cntr in contours:
                # Use bouding rectangle to find width and height of each FA to calculate area
                x,y,w,h = cv2.boundingRect(cntr)
                w = w*scale
                h = h*scale
                area = w*h
                # Write FAs that have an area that falls within established boundary into a new empty image
                if size_threshold_low <= area <= size_threshold_up:
                    cv2.drawContours(binary_img, [cntr], 0, (255, 0, 0), -1)
            # Apply laplacian operator to binary FA image to obtain segmented FA image
            kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
            seg_img = ndimage.convolve(binary_img, kernel_laplace, mode='constant')
            seg_img = seg_img * 255
            ret, seg_img = cv2.threshold(seg_img,10,255,cv2.THRESH_BINARY)
            # Write processed slice into 2D projection
            new_stack += binary_img
        return new_stack
    
    
    def membrane_processing(img: int):
        '''
        Returns numpy array for the segmented membrane image from the raw membrane image.
        
            Parameters:
                    img: A numpy array of stack containing raw membrane
            
            Returns:
                    output_img[index]: A numpy array of the slice containing membrane outline
        '''
        
        output_img = np.zeros((len(img), 256, 256))
        memb_slice = []
        for i in range(len(img)):
            # Read image and extract membrane channel to binarize
            ret, bw_img = cv2.threshold(img[i,:,:,0],10,255,cv2.THRESH_BINARY)
            # Place binary membrane channel into a new empty image
            res = np.zeros((256, 256, 3))
            res[:,:,0] = bw_img
            gray = rgb2gray(res)
            # Apply laplacian operator to binary membrane image to obtain segmented membrane image
            kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
            out_l = ndimage.convolve(gray, kernel_laplace, mode='constant')
            out_l = out_l * 255
            ret, seg_img = cv2.threshold(out_l,10,255,cv2.THRESH_BINARY)
            output_img[i] = seg_img
            xs = range(seg_img.shape[0])
            ys = range(seg_img.shape[1])
            indices = np.array(list(product(xs, ys)))
            memb_slice.append(len(indices))
        index = memb_slice.index(max(memb_slice))
        return output_img[index]
    
    
    # Calculation of distribution (d) measurement
    
    # Apply membrane processing function to raw ground truth stack
    membrane = membrane_processing(ground_truth)
    # Obtain x-y coordinates for membrane outline from segmented membrane slice
    pixels = membrane.reshape(-1)
    xs = range(membrane.shape[0])
    ys = range(membrane.shape[1])
    indices = np.array(list(product(xs, ys)))
    index = pd.Series(pixels, name="pixels")
    df = pd.DataFrame({
        "xmem" : indices[:, 0],
        "ymem" : indices[:, 1]
    }, index=index)
    df = df[df.index != 0]
    df = df.reset_index()
    df = df.drop(columns=['pixels'])
    x_values = df['xmem'].values
    x_arr = np.reshape(x_values, (-1, 1))
    y_values = df['ymem'].values
    y_arr = np.reshape(y_values, (-1, 1))
    if len(x_values) > 0:
        # Obtain minimum and maximum x and y values
        min_x = min(x_values)
        max_x = max(x_values)
        min_y = min(y_values)
        max_y = max(y_values)
        coordinates = np.hstack((x_arr, y_arr))
        # Extract bounds for membrane outline into a list
        coord_bound = []
        for x in range(min_x, max_x):
            new_coord = []
            for i in range(len(coordinates)):
                if coordinates[i][0] == x:
                    new_coord.append(coordinates[i][1])
            if len(new_coord) > 0:
                y_min = min(new_coord)
                y_max = max(new_coord)
                coord_bound.append([x, y_min, y_max])

    # Apply fa processing function to raw ground truth stack using size thresholds and scale defined in dpm function
    fa_binary_truth = fa_processing_stack(ground_truth, size_threshold_low, size_threshold_up, scale)
    # Segment FAs from 2D projection resulting from fa processing function
    kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
    seg_img = ndimage.convolve(fa_binary_truth, kernel_laplace, mode='constant')
    seg_img = seg_img * 255
    ret, seg_img_truth = cv2.threshold(seg_img,10,255,cv2.THRESH_BINARY)
    # Extract x-y coordinates for centroid of each ground truth FA site into a list
    labels= measure.label(seg_img_truth, background=0)
    bg_label = labels[0,0] 
    labels[labels==bg_label] = 0
    props = measure.regionprops(labels)
    centroids = np.zeros(shape=(len(np.unique(labels)) - 1,2))
    for i,prop in enumerate(props):
        my_centroid = prop.centroid
        centr = [0, 0]
        centr[0] = my_centroid[0]
        centr[1] = my_centroid[1]
        centroids[i,:]= centr
    
    # Train a k-means clustering algorithm using ground truth centroid coordinate data
    # If FA number is less than 5, match number of clusters to number of FAs
    if 0 < len(centroids) < 5:
        kmeans = KMeans(n_clusters = len(centroids), init = 'k-means++', random_state = 42).fit(centroids)
    # If FA number is greater or equal to 5, number of clusters is 5
    elif len(centroids) >= 5:
        kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42).fit(centroids)

    # Apply fa processing function to raw prediction stack using size thresholds and scale defined in dpm function
    fa_binary_pred = fa_processing_stack(prediction, size_threshold_low, size_threshold_up, scale)
    # Segment FAs from 2D projection resulting from fa processing function
    kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
    seg_img = ndimage.convolve(fa_binary_pred, kernel_laplace, mode='constant')
    seg_img = seg_img * 255
    ret, seg_img_pred = cv2.threshold(seg_img,10,255,cv2.THRESH_BINARY)
    # Extract x-y coordinates for centroid of each predicted FA site into a list
    labels= measure.label(seg_img_pred, background=0)
    bg_label = labels[0,0] 
    labels[labels==bg_label] = 0
    props = measure.regionprops(labels)
    centroids2 = np.zeros(shape=(len(np.unique(labels)) - 1,2))
    for i,prop in enumerate(props):
        my_centroid = prop.centroid
        centr = [0, 0]
        centr[0] = my_centroid[0]
        centr[1] = my_centroid[1]
        centroids2[i,:]= centr
    # Iterate through all predicted FA centroids and only keep centroids that are inside of membrane outline in a new list
    centroids_pred = []
    for i in range(len(centroids2)):
        if len(x_values) > 0:
            for j in range(len(coord_bound)):
                if int(centroids2[i][0]) == coord_bound[j][0] and coord_bound[j][1] <= int(centroids2[i][1]) <= coord_bound[j][2]:
                    centroids_pred.append([centroids2[i][0], centroids2[i][1]])
        else:
            centroids_pred = centroids2
    # Calculate number of dropped FAs and use as penalizing factor
    dropped_number = len(centroids2) - len(centroids_pred)
    if len(centroids2) > 0:
        factor = 1 - (dropped_number/len(centroids2))
    else:
        factor = 0
    # Use trained k-means clustering algorithm to predict to which cluster each predicted FA belongs to
    if len(centroids2) > 0 and len(centroids) > 0:
        pred_clusters = kmeans.predict(centroids2)
        # Count number of predicted FAs on each cluster
        unique_pred, counts_pred = np.unique(pred_clusters, return_counts=True)
        # Count number of ground truth FAs on each cluster
        truth_clusters = kmeans.labels_
        unique_truth, counts_truth = np.unique(truth_clusters, return_counts=True)
        # If a cluster has zero FAs, add zero to the counts list
        clusters = []
        if len(counts_truth) != len(counts_pred):
            for j in range(len(counts_truth)-len(counts_pred)):
                counts_pred = np.append(counts_pred, 0)
        # Calculate the ratio of predicted FAs to ground truth FAs on each cluster
        for i in range(len(counts_truth)):
            cluster_ind = (min(counts_truth[i], counts_pred[i]))/(max(counts_truth[i], counts_pred[i]))
            clusters.append(cluster_ind)
        # Calculate d measurement by taking average of ratios for each cluster and multiply by penalizing factor
        d = statistics.mean(clusters)
    elif len(centroids2) == 0 and len(centroids) == 0:
        d = 1
    else:
        d = 0
    
    # Calculation of shape/size (s) and angle (a) measurement
    
    # Use 2D FA projection obtained previously to draw bounding rectangles around each FA in prediction
    segmented_fa = fa_binary_pred
    segmented_fa_gray = rgb2gray(segmented_fa)
    gray = segmented_fa_gray*255
    gray = gray.astype(np.uint8)
    # Find contours for each predicted FA site
    ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    pred_img = np.zeros((256,256))
    rect_coord = []
    rect_dim = []
    rect_angle = []
    # Iterate through all predicted FA sites
    for cntr in contours:
        # Use bouding rectangle to obtain dimensions and coordinates of each FA site
        x,y,w,h = cv2.boundingRect(cntr)
        rect_coord.append([x,y])
        rect_dim.append([w,h])
        # Draw rectangles in a new image
        cv2.rectangle(pred_img, (x, y), (x+w, y+h), (255, 0, 0), -1)
        # Use minimum area rectangle to obtain angle of orientation of each FA site
        center, dim, angle = cv2.minAreaRect(cntr)
        rect_angle.append(angle)
    
    # Repeat same procedure as above to obtain dimensions, coordinates and angle of each ground truth FA site
    segmented_fa_truth = fa_binary_truth
    segmented_fa_truth_gray = rgb2gray(segmented_fa_truth)
    gray = segmented_fa_truth_gray*255
    gray = gray.astype(np.uint8)
    ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    new_img = np.zeros((256,256))
    rect_coord_truth = []
    rect_dim_truth = []
    rect_angle_truth = []
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        rect_coord_truth.append([x,y])
        rect_dim_truth.append([w,h])
        cv2.rectangle(new_img, (x, y), (x+w, y+h), (255, 0, 0), -1)
        center, dim, angle = cv2.minAreaRect(cntr)
        rect_angle_truth.append(angle)
    
    # Create new image to match location of FA sites in ground truth and prediction
    ground_truth_centered_img = np.zeros((256,256))
    angle_scores = []
    # Iterate through all predicted FA sites
    for i in range(len(rect_coord)):
        # Obtain centroid coordinates for FA site
        x_center = rect_coord[i][0] + rect_dim[i][0]//2
        y_center = rect_coord[i][1] + rect_dim[i][1]//2
        # Obtain nearest neighbor in the ground truth FA sites to the predicted FA site
        neighbors = get_neighbors(rect_coord_truth, rect_coord[i], 1)
        for neighbor in neighbors:
            # Extract nearest neighbor position in the ground truth FA sites list
            for j in range(len(rect_coord_truth)):
                if neighbor[0] == rect_coord_truth[j][0] and neighbor[1] == rect_coord_truth[j][1]:
                    # Obtain dimensions and centroid coordinates for nearest neighbor ground truth FA site
                    w = rect_dim_truth[j][0]
                    h = rect_dim_truth[j][1]
                    x = x_center - w//2
                    y = y_center - h//2
                    # Write new image with ground truth FA sites located at same position as predicted FA sites
                    cv2.rectangle(ground_truth_centered_img, (x, y), (x+w, y+h), (255, 0, 0), -1)
                    # Obtain angle score by normalizing angle values to a range of 0-90 degrees
                    if rect_angle[i] > 90:
                        angle_pred = 180 - rect_angle[i]
                    elif -90 <= rect_angle[i] < 0:
                        angle_pred = abs(rect_angle[i])
                    elif rect_angle[i] < -90:
                        angle_pred = 180 - abs(rect_angle[i])
                    else:
                        angle_pred = rect_angle[i]
                    if rect_angle_truth[j] > 90:
                        angle_truth = 180 - rect_angle_truth[j]
                    elif -90 <= rect_angle_truth[j] < 0:
                        angle_truth = abs(rect_angle_truth[j])
                    elif rect_angle_truth[j] < -90:
                        angle_truth = 180 - abs(rect_angle_truth[j])
                    else:
                        angle_truth = rect_angle_truth[j]
                    # Calculate deviation between predicted and ground truth FA site angle
                    angle_dev = abs(angle_pred - angle_truth)
                    angle_ind = 1 - (angle_dev/90)
                    angle_scores.append(angle_ind)
    
    # Compare ground truth centered image to predicted image
    y_true = ground_truth_centered_img.flatten()
    y_true = y_true/255
    y_pred = pred_img.flatten()
    y_pred = y_pred/255
    # Compute true positives, false negatives, and false positives
    tp = 0
    fn = 0
    fp = 0
    for k in range(len(y_true)):
        if y_true[k]==1 and y_pred[k]==1:
            tp += 1
        elif y_true[k]==1 and y_pred[k]==0:
            fn += 1
        elif y_true[k]==0 and y_pred[k]==1:
            fp += 1
    # Calculate precision and recall
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    # Calculate s measurement using F1 score
    s = (2*precision*recall)/(precision+recall)
    # Calculate a measurement by taking averge of angle scores across all FA sites
    a = statistics.mean(angle_scores)

    # Calculate dpm score using defined weights and obtained d, s, and a measurements
    if d == 1 or s == 1 or a == 1:
        dpm = 1
    elif d == 0 or s == 0 or a == 0:
        dpm = 0
    else:
        dpm = (weight_d*d)+(weight_s*s)+(weight_a*a)
    
    
    return dpm, d, s, a, fa_binary_truth, fa_binary_pred

In [ ]:
def fa_nd(
    nucleus_truth: int, nucleus_prediction: int,
    fa_truth: int, fa_prediction: int,
    size_threshold_low: int, size_threshold_up: int, scale: float):
    '''
    Returns the FA-Nucleus Distribution (FA-ND) score for subcellular distribution similarity between true cell and digital twin.
    
            Parameters:
                    nucleus_truth: A numpy array of stack containing ground truth nucleus
                    nucleus_prediction: A numpy array of stack containing predicted nucleus
                    fa_truth: A numpy array of stack containing ground truth FA sites
                    fa_prediction: A numpy array of stack containing predicted FA sites
                    size_threshold_low: An integer value for lower bound size threshold; any FAs with an area less than this value are dropped
                    size_threshold_up: An integer value for upper bound size threshold; any FAs with an area higher than this value are dropped
                    scale: A float value for image scale (microns/pixel)
            
            Returns:
                    fa_nd: A numpy float value for the FA-Nucleus Distribution score
    '''
    
    def euclidean_distance(row1: slice, row2: slice):
        '''
        Returns the euclidean distance between two coordinate points.
        
            Parameters:
                    row1: A slice of a list containing a single coordinate point
                    row2: A slice of a list containing another coordinate point
            
            Returns:
                    sqrt(distance): A float value for the euclidean distance
        '''
        distance = 0.0
        for i in range(len(row1)-1):
            distance += (row1[i] - row2[i])**2
        return sqrt(distance)

    
    def nucl_centroid(img: int):
        '''
        Returns x-y coordinates of nucleus centroid from raw nucleus image.

            Parameters:
                    img: A numpy array of the image containing raw nucleus

            Returns:
                    centroids: Nucleus centroid x-y coordinates
        '''
        # Convert nucleus image into 8-bit image
        nucleus = np.uint8(img)
        # Binarize nucleus image and place into empty image
        ret, bw_img = cv2.threshold(nucleus[20,:,:,2],20,255,cv2.THRESH_BINARY)
        res = np.zeros((256, 256, 3))
        res[:,:,2] = bw_img
        gray = rgb2gray(res)
        # Apply laplacian operator to binary nucleus image to obtain nucleus outline image
        kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
        out_l = ndimage.convolve(gray, kernel_laplace, mode='constant')
        out_l = out_l * 255
        ret, seg_img = cv2.threshold(out_l,10,255,cv2.THRESH_BINARY)
        # Extract x-y coordinates of nucleus centroid from nucleus outline image
        labels= measure.label(seg_img, background=0)
        bg_label = labels[0,0] 
        labels[labels==bg_label] = 0
        props = measure.regionprops(labels)
        centroids = np.zeros(shape=(len(np.unique(labels)) - 1,2))
        for i,prop in enumerate(props):
            my_centroid = prop.centroid
            centr = [0, 0]
            centr[0] = my_centroid[0]
            centr[1] = my_centroid[1]
            centroids[i,:]= centr
        return centroids

    
    def fa_centroids(img: int, size_threshold_low: int, size_threshold_up: int, scale: float):
        '''
        Returns x-y coordinates of each FA site from raw FA image.

            Parameters:
                    img: A numpy array of the image containing raw FA
                    size_threshold_low: An integer value for lower bound size threshold; any FAs with an area less than this value are dropped
                    size_threshold_up: An integer value for upper bound size threshold; any FAs with an area higher than this value are dropped
                    scale: A float value for image scale (microns/pixel)

            Returns:
                    centroids: FA sites centroids x-y coordinates
        '''
        # Read FA stack
        stack = img
        new_stack = np.zeros((256, 256), dtype=np.uint8)
        # Iterate through each slice in FA stack
        for i in range(len(stack)):
            img = stack[i]
            res = np.zeros((256, 256, 3))
            res[:,:,1] = img[:,:,1]
            # Apply a top hat filter with 3x3 rectangular kernel and place filtered image into a new empty image
            filterSize =(3, 3)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize)
            tophat_img = cv2.morphologyEx(res[:,:,1], cv2.MORPH_TOPHAT, kernel)        
            processed_img = res
            processed_img[:,:,1] = tophat_img
            # Binarize filtered FA image
            ret, bw_img = cv2.threshold(processed_img,10,255,cv2.THRESH_BINARY)
            gray = rgb2gray(bw_img)
            gray = gray * 255
            gray = gray.astype(np.uint8)
            ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
            # Find the contours of each FA
            contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            binary_img = np.zeros((256,256), dtype=np.uint8)
            # Iterate through each contour
            for cntr in contours:
                # Use bouding rectangle to find width and height of each FA to calculate area
                x,y,w,h = cv2.boundingRect(cntr)
                w = w*scale
                h = h*scale
                area = w*h
                # Write FAs that have an area within the established thresholds into a new empty image
                if size_threshold_low <= area <= size_threshold_up:
                    cv2.drawContours(binary_img, [cntr], 0, (255, 0, 0), -1)
            # Add binary FA slice into empty image to create 2D projection of FA sites
            new_stack += binary_img
        # Apply Laplacian operator to segment FA sites from 2D projection
        kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
        seg_img = ndimage.convolve(new_stack, kernel_laplace, mode='constant')
        seg_img = seg_img * 255
        # Extract x-y coordinates of FA centroids from segmented FA image
        ret, seg_img = cv2.threshold(seg_img,10,255,cv2.THRESH_BINARY)
        labels= measure.label(seg_img, background=0)
        bg_label = labels[0,0] 
        labels[labels==bg_label] = 0
        props = measure.regionprops(labels)
        centroids = np.zeros(shape=(len(np.unique(labels)) - 1,2))
        for i,prop in enumerate(props):
            my_centroid = prop.centroid
            centr = [0, 0]
            centr[0] = my_centroid[0]
            centr[1] = my_centroid[1]
            centroids[i,:]= centr
        return centroids
    
    
    # Extract nucleus and FAs centroids from ground truth
    fa_centroids_truth = fa_centroids(fa_truth, size_threshold_low, size_threshold_up, scale)
    nucl_centroid_truth = nucl_centroid(nucleus_truth)
    # Calculate Eucleidean distance between nucleus centroid and centroid of each FA
    distance_truth = []
    for i in range(len(fa_centroids_truth)):
        distance_truth.append(euclidean_distance(nucl_centroid_truth[0], fa_centroids_truth[i]))
    # Extract nucleus and FAs centroids from prediction
    fa_centroids_pred = fa_centroids(fa_prediction, size_threshold_low, size_threshold_up, scale)
    nucl_centroid_pred = nucl_centroid(nucleus_prediction)
    # Calculate Euclidean distance between nucleus centroid and centroid of each FA
    distance_pred = []
    for i in range(len(fa_centroids_pred)):
        distance_pred.append(euclidean_distance(nucl_centroid_pred[0], fa_centroids_pred[i]))
    
    # Calculate sum of Euclidean distances in ground truth and prediction
    total_distance_truth = sum(distance_truth)
    total_distance_pred = sum(distance_pred)
    # Calculate FA-Nucleus Distribution score as ratio between sum of distances in ground truth and prediction
    fa_nd = min(total_distance_truth, total_distance_pred)/max(total_distance_truth, total_distance_pred)
    
    return fa_nd

In [ ]:
# Directory where 3D cells are located
test_dir = '3D_cells'
cells = os.listdir(test_dir)
# Store directory paths for each individual 3D cell in empty list
cell_folders = []
for i in range(len(cells)):
    cell_folders.append(os.path.join(test_dir, 'cell{}'.format(i+1)))
# Create empty list to store dpm, d, s, a, and pearson values for later calculation of average of each metric
dpm_avg = []
d_avg = []
s_avg = []
a_avg = []
pearson_avg = []
# Create empty list to store global and FA-Nucleus distribution score
global_avg = []
fa_nd_avg = []
for folder in cell_folders:
    for eval_folder in os.listdir(folder):
        # Evaluate FA predictions located on fa subfolder using DPM
        if eval_folder == 'fa':
            img_folder = os.path.join(folder, eval_folder)
            for image in os.listdir(img_folder):
                if image == 'ground_truth.tiff':
                    ground_truth = os.path.join(img_folder, image)
                    ground_truth = io.imread(ground_truth)
                    fa_truth = np.uint8(ground_truth)
                    fa_truth_processed = np.uint8(ground_truth)
                elif image == 'prediction.tiff':
                    prediction = os.path.join(img_folder, image)        
                    prediction = io.imread(prediction)
                    fa_prediction = np.uint8(prediction)
                    fa_prediction_processed = np.uint8(prediction)
            dpm = dpm_3d(fa_truth, fa_prediction, 1, 4, 0.7, 0.33, 0.33, 0.33)
            for i in range(len(fa_truth_processed)):
                fa_truth_processed[i,:,:,1] = cv2.bitwise_and(fa_truth_processed[i,:,:,1], fa_truth_processed[i,:,:,1], mask=dpm[4])
                fa_prediction_processed[i,:,:,1] = cv2.bitwise_and(fa_prediction_processed[i,:,0:256,1], fa_prediction_processed[i,:,:,1], mask=dpm[5])
            dpm_avg.append(dpm[0]*100)
            d_avg.append(dpm[1]*100)
            s_avg.append(dpm[2]*100)
            a_avg.append(dpm[3]*100)
            print(folder)
            print('FA: {}'.format(dpm[0]*100))
        # Evaluate Nucleus predictions located on nucl subfolder using Pearson correlation
        elif eval_folder == 'nucl':
            img_folder = os.path.join(folder, eval_folder)
            for image in os.listdir(img_folder):
                if image == 'ground_truth.tiff':
                    ground_truth_path = os.path.join(img_folder, image)
                    ground_truth = io.imread(ground_truth_path)
                    nucl_truth = ground_truth[:,:,:,2].flatten()
                    nucl_truth = np.uint8(nucl_truth)
                    membrane = ground_truth[:,:,:,0]
                    nucleus_truth = ground_truth
                    nucleus_truth = np.uint8(nucleus_truth)
                    nucleus_truth_processed = nucleus_truth
                    for i in range(len(nucleus_truth_processed)):
                        ret, thresh = cv2.threshold(nucleus_truth_processed[i], 100, 255, cv2.THRESH_TOZERO)
                        nucleus_truth_processed[i] = thresh
                elif image == 'prediction.tiff':
                    prediction_path = os.path.join(img_folder, image)
                    prediction = io.imread(prediction_path)
                    nucl_pred = prediction[:,:,:,2].flatten()
                    nucl_pred = np.uint8(nucl_pred)
                    nucleus_pred = prediction
                    nucleus_pred = np.uint8(nucleus_pred)
                    nucleus_pred_processed = nucleus_pred
                    for i in range(len(nucleus_pred_processed)):
                        ret, thresh = cv2.threshold(nucleus_pred_processed[i], 50, 255, cv2.THRESH_TOZERO)
                        nucleus_pred_processed[i] = thresh
            pearson_value = pearsonr(nucl_truth, nucl_pred)
            pearson_avg.append(pearson_value[0]*100)
            print('Nucl: {}'.format(pearson_value[0]*100))
    # Calculate Global and FA-ND score
    global_score = 0.5*pearson_value[0] + 0.5*dpm[0]
    global_avg.append(global_score*100)
    print('Global: {}'.format(global_score*100))
    fa_nd_score = fa_nd(nucleus_truth, nucleus_pred, fa_truth, fa_prediction, 1, 4, 0.7)
    fa_nd_avg.append(fa_nd_score*100)
    print('FA-ND: {}'.format(fa_nd_score*100))
    # Build digital twin using processed predicted Nucleus and FA sites and compare to true cell
    for eval_folder in os.listdir(folder):
        if eval_folder == 'visualize':
            img_folder = os.path.join(folder, eval_folder)
            panel = np.zeros((64, 256, 256*2+2, 3), dtype=np.uint8)
            panel[:,:,0:256,0] = membrane
            panel[:,:,256+2:256*2+2,0] = membrane
            panel[:,:,0:256,1] = fa_truth_processed[:,:,:,1]
            panel[:,:,256+2:256*2+2,1] = fa_prediction_processed[:,:,:,1]
            panel[:,:,0:256,2] = nucleus_truth_processed[:,:,:,2]
            panel[:,:,256+2:256*2+2,2] = nucleus_pred_processed[:,:,:,2]
            new_img = np.zeros((3, 64, 256, 256*2+2), dtype=np.uint8)
            new_img[0] = panel[:,:,:,1]
            new_img[1] = panel[:,:,:,2]
            new_img[2] = panel[:,:,:,0]
            io.imsave((os.path.join(img_folder, 'cell_comparison.tiff')), panel)
            io.imsave((os.path.join(img_folder, 'agave_cell_comparison.tiff')), new_img)
# Calculate mean and standard deviation across all cells
pearson_avg.append(statistics.mean(pearson_avg))
dpm_avg.append(statistics.mean(dpm_avg))
d_avg.append(statistics.mean(d_avg))
s_avg.append(statistics.mean(s_avg))
a_avg.append(statistics.mean(a_avg))
global_avg.append(statistics.mean(global_avg))
fa_nd_avg.append(statistics.mean(fa_nd_avg))
pearson_avg.append(statistics.stdev(pearson_avg))
dpm_avg.append(statistics.stdev(dpm_avg))
d_avg.append(statistics.stdev(d_avg))
s_avg.append(statistics.stdev(s_avg))
a_avg.append(statistics.stdev(a_avg))
global_avg.append(statistics.stdev(global_avg))
fa_nd_avg.append(statistics.stdev(fa_nd_avg))

In [ ]:
indices = []
for i in range(len(cells)):
    indices.append('Cell{}'.format(i+1))
indices.append('Average')
indices.append('Standard deviaton')
df = pd.DataFrame({
    "Nucleus (PCC)": pearson_avg,
    "FA (DPM)": dpm_avg,
    "FA (d)": d_avg,
    "FA (s)": s_avg,
    "FA (a)": a_avg,
    "Global": global_avg,
    "FA-ND": fa_nd_avg
}, index = indices)
print(df)

In [ ]:
df.to_csv('3D_cells_results.csv')